In [167]:
import pandas as pd
import numpy as np

## **Items**

In [168]:
item_columns = ['item id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
                'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item = pd.read_csv('../data/raw/ml-100k/u.item', sep='|', header=None, encoding='latin-1', 
                   names=item_columns)


video_release_date contains all NaN

In [169]:
item['video_release_date'].isna().sum() == len(item['video_release_date'])

True

Movie title, url and dates do not contain necessary information for recommendation

In [170]:
item.drop(['movie_title','release_date','video_release_date', 'IMDb_URL'], axis=1, inplace=True)
item

,item id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [171]:
item.isna().sum()

item id        0
unknown        0
Action         0
Adventure      0
Animation      0
Children's     0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
dtype: int64

## **Users**

In [172]:
user_columns = ['user id', 'age', 'gender', 'occupation', 'zip code']
user = pd.read_csv('../data/raw/ml-100k/u.user', sep='|', header=None, encoding='latin-1', names=user_columns)
user

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


zip code also have no sence in recommentaion

In [173]:
user.drop(['zip code'], axis=1, inplace=True)
user

,user id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other
...,...,...,...,...
938,939,26,F,student
939,940,32,M,administrator
940,941,20,M,student
941,942,48,F,librarian


In [174]:
user.isna().sum()

user id       0
age           0
gender        0
occupation    0
dtype: int64

Let's divide ages into groups, it may increase score, for example childs are more likely would watch cartoons(in our dataset genres:[Animation,	Children's])

In [175]:
import numpy as np

user['age_group'] = np.digitize(user['age'], bins=[0, 12, 18, 30, 45, float('inf')], right=True)

# Mapping the digitized values to age groups
age_group_mapping = {
    1: 'child',
    2: 'teenage',
    3: 'adults',
    4: 'middle age',
    5: 'old adults'
}

user['age_group'] = user['age_group'].map(age_group_mapping)
user.drop(['age'], axis=1, inplace=True)
user

,user id,gender,occupation,age_group
0,1,M,technician,adults
1,2,F,other,old adults
2,3,M,writer,adults
3,4,M,technician,adults
4,5,F,other,middle age
...,...,...,...,...
938,939,F,student,adults
939,940,M,administrator,middle age
940,941,M,student,adults
941,942,F,librarian,old adults


## **Data**

In [176]:
data_columns = ['user id', 'item id', 'rating', 'timestamp']
data = pd.read_csv('../data/raw/ml-100k/u.data', sep='\t', header=None, encoding='latin-1', names=data_columns)
data

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


Recommend only with high value rating

In [177]:
data = data.loc[data['rating'] > 3]
data

,user id,item id,rating,timestamp
5,298,474,4,884182806
7,253,465,5,891628467
11,286,1014,5,879781125
12,200,222,5,876042340
16,122,387,5,879270459
...,...,...,...,...
99988,421,498,4,892241344
99989,495,1091,4,888637503
99990,806,421,4,882388897
99991,676,538,4,892685437


no duplicates, and no user rated the same film many times

In [178]:
print(data.duplicated().sum())
print(data.duplicated(subset=['user id', 'item id']).sum())

0
0


In [179]:
merged = pd.merge(data, item, how='left', on='item id')
merged = pd.merge(merged, user, how='left', on='user id')
merged = merged.sort_values(by=["user id", 'timestamp'])
merged.reset_index(drop=True, inplace=True)
merged

,user id,item id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,gender,occupation,age_group
0,1,168,5,874965478,0,0,0,0,0,1,...,0,0,0,0,0,0,0,M,technician,adults
1,1,172,5,874965478,0,1,1,0,0,0,...,0,0,1,1,0,1,0,M,technician,adults
2,1,165,5,874965518,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M,technician,adults
3,1,156,4,874965556,0,0,0,0,0,0,...,0,0,0,0,1,0,0,M,technician,adults
4,1,196,5,874965677,0,0,0,0,0,0,...,0,0,0,0,0,0,0,M,technician,adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55370,943,1074,4,888640250,0,0,0,0,0,1,...,0,0,0,0,0,0,0,M,student,adults
55371,943,41,4,888640251,0,0,0,0,0,1,...,0,0,0,0,0,0,0,M,student,adults
55372,943,237,4,888692413,0,0,0,0,0,0,...,0,0,1,0,0,0,0,M,student,adults
55373,943,151,4,888692699,0,0,1,0,1,1,...,0,0,0,0,0,0,0,M,student,adults


Let's create feature that contains number of films rated before rating corresponding item

In [180]:
# Assuming 'user id' is the correct column name in your DataFrame
user_id_series = merged['user id']

def count_condition_optimized(row):
    user_id = row['user id']
    count = ((user_id_series == user_id) & (merged.index < row.name)).sum()
    return count

merged['n_rated'] = merged.apply(count_condition_optimized, axis=1)

In [181]:
merged

,user id,item id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,gender,occupation,age_group,n_rated
0,1,168,5,874965478,0,0,0,0,0,1,...,0,0,0,0,0,0,M,technician,adults,0
1,1,172,5,874965478,0,1,1,0,0,0,...,0,1,1,0,1,0,M,technician,adults,1
2,1,165,5,874965518,0,0,0,0,0,0,...,0,0,0,0,0,0,M,technician,adults,2
3,1,156,4,874965556,0,0,0,0,0,0,...,0,0,0,1,0,0,M,technician,adults,3
4,1,196,5,874965677,0,0,0,0,0,0,...,0,0,0,0,0,0,M,technician,adults,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55370,943,1074,4,888640250,0,0,0,0,0,1,...,0,0,0,0,0,0,M,student,adults,91
55371,943,41,4,888640251,0,0,0,0,0,1,...,0,0,0,0,0,0,M,student,adults,92
55372,943,237,4,888692413,0,0,0,0,0,0,...,0,1,0,0,0,0,M,student,adults,93
55373,943,151,4,888692699,0,0,1,0,1,1,...,0,0,0,0,0,0,M,student,adults,94


more features! Now let's for each row calculate how many filmes of each genre the user watched, before rating this film

In [183]:
columns = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime',
           'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
           'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Define a function to calculate the sum for a specific column
def rated_genres_optimized(row, column):
    user_id = row['user id']
    return merged.loc[(merged['user id'] == user_id) & (merged.index < row.name), column].sum()

# Create new columns with the suffix "_sum"
for column in columns:
    merged[column + '_sum'] = merged.apply(lambda row: rated_genres_optimized(row, column), axis=1)


In [ ]:
merged

,user id,item id,rating,timestamp,unknown,Action,Adventure,Animation,Children's,Comedy,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,gender,occupation,age_group,n_rated
0,1,168,5,874965478,0,0,0,0,0,0,...,0,0,0,0,0,0,M,technician,adults,0
1,1,172,5,874965478,0,0,0,0,0,1,...,0,0,0,0,0,0,M,technician,adults,1
2,1,165,5,874965518,0,1,1,0,0,1,...,0,1,1,0,1,0,M,technician,adults,2
3,1,156,4,874965556,0,1,1,0,0,1,...,0,1,1,0,1,0,M,technician,adults,3
4,1,196,5,874965677,0,1,1,0,0,1,...,0,1,1,1,1,0,M,technician,adults,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55370,943,1074,4,888640250,0,41,20,1,2,20,...,2,15,8,26,10,2,M,student,adults,91
55371,943,41,4,888640251,0,41,20,1,2,21,...,2,15,8,26,10,2,M,student,adults,92
55372,943,237,4,888692413,0,41,20,1,2,22,...,2,15,8,26,10,2,M,student,adults,93
55373,943,151,4,888692699,0,41,20,1,2,22,...,2,16,8,26,10,2,M,student,adults,94


In [184]:
merged.drop(['timestamp'], axis=1, inplace=True)

In [185]:
merged

,user id,item id,rating,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy_sum,Film-Noir_sum,Horror_sum,Musical_sum,Mystery_sum,Romance_sum,Sci-Fi_sum,Thriller_sum,War_sum,Western_sum
0,1,168,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,172,5,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,165,5,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
3,1,156,4,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,1,0
4,1,196,5,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55370,943,1074,4,0,0,0,0,0,1,0,...,0,0,8,2,2,15,8,26,10,2
55371,943,41,4,0,0,0,0,0,1,0,...,0,0,8,2,2,15,8,26,10,2
55372,943,237,4,0,0,0,0,0,0,0,...,0,0,8,2,2,15,8,26,10,2
55373,943,151,4,0,0,1,0,1,1,0,...,0,0,8,2,2,16,8,26,10,2


In [186]:
merged.to_csv('../data/interim/100k_clean.csv')